## Code to obtain the cost of a dollar in pesos from different platforms
- Banxico, using Banxico API
- Yahoo Finance, using python lib
- eldolar.info using Web Scraping

## Test for Banxico using the API

In [5]:
import requests
from datetime import date, timedelta
import pandas as pd
import os
from dotenv import load_dotenv  

# Loading Token from env 
load_dotenv()
TOKEN = os.environ["BANXICO_API_TOKEN"]

ID = "SF43718"  
hoy = date.today()
ini = (hoy - timedelta(days=15)).isoformat()
fin = hoy.isoformat()

url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{ID}/datos/{ini}/{fin}"
headers = {"Bmx-Token": TOKEN, "Accept": "application/json"}

r = requests.get(url, headers=headers, timeout=20)
r.raise_for_status()
datos = r.json()["bmx"]["series"][0]["datos"]

# Filters most recet value
datos_validos = [d for d in datos if d["dato"] not in ("N/E", "", None)]
ultimo = max(datos_validos, key=lambda d: d["fecha"]) if datos_validos else None

# Simple print of the last value
print(ultimo)  # {'fecha': '2025-09-15', 'dato': 'XX.XXXX'}

{'fecha': '19/09/2025', 'dato': '18.3892'}


## Test for Yahoo Finance using the python capability

In [6]:
import yfinance as yf
import pandas as pd

# Yahoo Finance symbol for USD/MXN
symbol = "MXN=X"

# Download ~15 days of data and keep the last 10 valid rows
df = yf.download(symbol, period="15d", interval="1d", progress=False)

# Keep only the Close column (daily closing price)
df_fx = df[["Close"]].dropna().tail(10).copy()

# Rename the column to something clearer
df_fx.rename(columns={"Close": "USD_MXN"}, inplace=True)

# Extra: compute daily % return and 7-day moving average
df_fx["ret_diario_%"] = df_fx["USD_MXN"].pct_change().mul(100).round(4)
df_fx["MA_7d"] = df_fx["USD_MXN"].rolling(7).mean().round(6)

# Resulting DataFrame
print(df_fx)

/tmp/ipykernel_6018/184073625.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="15d", interval="1d", progress=False)


Price         USD_MXN ret_diario_%      MA_7d
Ticker          MXN=X                        
Date                                         
2025-09-09  18.652870          NaN        NaN
2025-09-10  18.623501      -0.1575        NaN
2025-09-11  18.580799      -0.2293        NaN
2025-09-12  18.454000      -0.6824        NaN
2025-09-15  18.446989      -0.0380        NaN
2025-09-16  18.361521      -0.4633        NaN
2025-09-17  18.285801      -0.4124  18.486497
2025-09-18  18.307800       0.1203  18.437202
2025-09-19  18.364201       0.3081  18.400159
2025-09-22  18.420200       0.3049  18.377216


## Web Scrapping version from eldolar.info

In [7]:
import requests, re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date

# URL from ElDolar.info only for the current day
URL = "https://www.eldolar.info/es-MX/mexico/dia/hoy"
r = requests.get(URL, headers={"User-Agent":"Mozilla/5.0"}, timeout=20)
r.raise_for_status()

soup = BeautifulSoup(r.text, "lxml")
text = soup.get_text(" ", strip=True)

# Regex to get the line from the web page text "1 Dólar = XX.XXXX Pesos"
m = re.search(r"1\s+Dólar\s*=\s*([\d\.]+)\s*Pesos", text)
promedio = float(m.group(1)) if m else None

# Pandas dataframe creation
df_eldolar = pd.DataFrame({
    "USD_MXN": [promedio],
})
df_eldolar["ret_diario_%"] = df_eldolar["USD_MXN"].pct_change().mul(100).round(4)
df_eldolar["MA_7d"] = df_eldolar["USD_MXN"].rolling(7).mean().round(6)
df_eldolar["fuente"] = "eldolar.info"

# We add current day to the dataframe
df_eldolar.index = [pd.to_datetime(date.today())]

print(df_eldolar)

            USD_MXN  ret_diario_%  MA_7d        fuente
2025-09-21  18.3072           NaN    NaN  eldolar.info


## Merge of all codes in a single Dataframe

In [9]:
import requests
from datetime import date, timedelta
import pandas as pd
import yfinance as yf
import os  

# Imports for web scraping
import re
from bs4 import BeautifulSoup

# =====================
# 1) Banxico (SIE-API)
# =====================
TOKEN = os.environ["BANXICO_API_TOKEN"]
ID = "SF43718"               
today = date.today()
start = (today - timedelta(days=15)).isoformat()
end = today.isoformat()

url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{ID}/datos/{start}/{end}"
headers = {"Bmx-Token": TOKEN, "Accept": "application/json"}

r = requests.get(url, headers=headers, timeout=20)
r.raise_for_status()
serie = r.json()["bmx"]["series"][0]

df_bmx = pd.DataFrame(serie["datos"])
# Cleaning step
df_bmx = df_bmx[df_bmx["dato"].notna() & (df_bmx["dato"] != "N/E")].copy()
df_bmx["USD_MXN"] = df_bmx["dato"].astype(float)
df_bmx["fecha"] = pd.to_datetime(df_bmx["fecha"], dayfirst=True)
df_bmx = df_bmx.sort_values("fecha").set_index("fecha")[["USD_MXN"]]

# Metrics
df_bmx["ret_diario_%"] = df_bmx["USD_MXN"].pct_change().mul(100).round(4)
df_bmx["MA_7d"] = df_bmx["USD_MXN"].rolling(7).mean().round(6)
df_bmx["source"] = "banxico"

# =====================
# 2) Yahoo Finance
# =====================
symbol = "MXN=X"
df_yf_raw = yf.download(symbol, period="15d", interval="1d", progress=False)

# It may return a MultiIndex; squeeze to a Series and normalize
close = df_yf_raw["Close"].squeeze()
df_yf = pd.DataFrame({"USD_MXN": close}).dropna().copy()
df_yf.index.name = "fecha"

# Metrics
df_yf["ret_diario_%"] = df_yf["USD_MXN"].pct_change().mul(100).round(4)
df_yf["MA_7d"] = df_yf["USD_MXN"].rolling(7).mean().round(6)
df_yf["source"] = "yahoo"

# If you want exactly the 10 most recent days per source:
df_bmx_10 = df_bmx.tail(10).copy()
df_yf_10 = df_yf.tail(10).copy()

# ===============================
# 3) ElDolar.info (web-scraping)
#    - Scrapes today's average USD/MXN printed in the HTML.
#    - Kept as a single-row DataFrame with today's date as index.
# ===============================
def fetch_eldolar_today() -> pd.DataFrame:
    url_eldolar = "https://www.eldolar.info/es-MX/mexico/dia/hoy"
    resp = requests.get(url_eldolar, headers={"User-Agent": "Mozilla/5.0"}, timeout=20)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "lxml")
    text = soup.get_text(" ", strip=True)

    # Look for a string like: "1 Dólar = XX.XXXX Pesos"
    m = re.search(r"1\s+Dólar\s*=\s*([\d\.]+)\s*Pesos", text)
    if not m:
        # If the pattern is not found, raise an informative error
        raise RuntimeError("Could not extract today's average from ElDolar.info.")

    # Pandas dataframe creation
    value = float(m.group(1))
    df_ed = pd.DataFrame({"USD_MXN": [value]})
    df_ed.index = [pd.to_datetime(date.today())]
    df_ed.index.name = "fecha"

    # Compute the same metrics to match the schema
    df_ed["ret_diario_%"] = df_ed["USD_MXN"].pct_change().mul(100).round(4)
    df_ed["MA_7d"] = df_ed["USD_MXN"].rolling(7).mean().round(6)
    df_ed["source"] = "eldolar.info"
    return df_ed

# Try to fetch ElDolar.info; if it fails, proceed without it
try:
    df_ed_today = fetch_eldolar_today()
except Exception as e:
    print(f"[WARN] ElDolar.info scraping failed: {e}")
    df_ed_today = pd.DataFrame(columns=["USD_MXN", "ret_diario_%", "MA_7d", "source"])

# =====================
# 4) Combine all dataframes (long format)
# =====================
cols = ["USD_MXN", "ret_diario_%", "MA_7d", "source"]
df_long = pd.concat(
    [df_bmx_10[cols], df_yf_10[cols], df_ed_today[cols]],
    axis=0
).sort_index()

print("\n=== Long format (all sources stacked) ===")
print(df_long.tail(15))

# =====================
# 5) Wide view
#    - Price by source: columns USD_MXN_banxico / USD_MXN_yahoo / USD_MXN_eldolar
#    - You can also pivot returns/MA if needed (will have a MultiIndex)
# =====================
prices_wide = (
    df_long.reset_index()
           .pivot(index="fecha", columns="source", values="USD_MXN")
           .rename_axis(None, axis=1)
           .rename(columns={
               "banxico": "USD_MXN_banxico",
               "yahoo": "USD_MXN_yahoo",
               "eldolar.info": "USD_MXN_eldolar"
           })
)

print("\n=== Wide view (prices by source) ===")
print(prices_wide.tail(10))

# Pivot EVERYTHING (price, return, MA) into columns by source (MultiIndex)
everything_wide = (
    df_long.reset_index()
           .melt(id_vars=["fecha","source"], var_name="metric", value_name="value")
           .pivot_table(index="fecha", columns=["metric","source"], values="value", aggfunc="first")
           .sort_index(axis=1)
)
print("\n=== Wide view with metrics (MultiIndex) ===")
print(everything_wide.tail(10))

/tmp/ipykernel_6018/1452689740.py:43: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_yf_raw = yf.download(symbol, period="15d", interval="1d", progress=False)



=== Long format (all sources stacked) ===
              USD_MXN  ret_diario_%      MA_7d        source
fecha                                                       
2025-09-11  18.528700       -0.3405        NaN       banxico
2025-09-11  18.580799       -0.2293  18.677804         yahoo
2025-09-12  18.454000       -0.6824  18.638562         yahoo
2025-09-12  18.475700       -0.2860        NaN       banxico
2025-09-15  18.363500       -0.6073        NaN       banxico
2025-09-15  18.446989       -0.0380  18.601474         yahoo
2025-09-16  18.361521       -0.4633  18.549434         yahoo
2025-09-17  18.325700       -0.2058  18.510843       banxico
2025-09-17  18.285801       -0.4124  18.486497         yahoo
2025-09-18  18.361000        0.1926  18.468843       banxico
2025-09-18  18.307800        0.1203  18.437202         yahoo
2025-09-19  18.389200        0.1536  18.433686       banxico
2025-09-19  18.364201        0.3081  18.400159         yahoo
2025-09-21  18.307400           NaN       